# I. Setup pour l'analyse

In [2]:
#import
from sqlalchemy import text, create_engine
import pandas as pd
import numpy as np

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, LabelEncoder
from sklearn.pipeline import Pipeline

from unidecode import unidecode

from textblob import TextBlob, Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

# modèle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import pickle

from utils import data_preproc 

Import et preproc dataset

In [78]:
# def data_preproc(data):
#     #garde seulement lignes où certificate_eligible et grade ne sont pas null
#     data.dropna(subset=['certificate_eligible', 'grade'], inplace = True)

#     # vire colonne inutile total_length
#     data = data.drop("total_length", axis=1)

#     # encode le label
#     data['certificate_eligible'] = LabelEncoder().fit_transform(data['certificate_eligible'])

#     #prépare le corpus pour textblob
#     data["corpus"]= data["corpus"].str.lower()
#     # envlever les accents
#     unidec_corpus = []
#     for (i, row) in data.iterrows():
#         truc = unidecode(row['corpus'])
#         # print(f"user: {row['user']} course :{row['course_id']} new_corp:{truc[:25]}")
#         unidec_corpus.append(truc)

#     data["corpus"] = unidec_corpus

#     tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

#     polarity = []
#     subjectivity = []

#     for (i, row) in data.iterrows():
#         # if i>10: quit()
#         blob = tb(row['corpus'])
#         # print(f"user: {row['user']} course :{row['course_id']} sentiment:{blob.sentiment}")
#         polarity.append(blob.sentiment[0])
#         subjectivity.append(blob.sentiment[1])

#     #update le dataset avec les nouvelles valeurs
#     data['polarity']=polarity
#     data['subjectivity']=subjectivity

#     return(data)

In [2]:
data = pd.read_csv("data/dataset.csv", sep = ",")
data = data_preproc(data)
data.head()

,user,gender,country,level_of_education,course_id,nb_threads,nb_comments,corpus,delai_1er_post,grade,certificate_eligible,polarity,subjectivity
10,-Luca-,NaN,NaN,NaN,course-v1:MinesTelecom+04017+session03,0,1,"bonjour, je m'appelle luca, je suis consultant...",5,0.38,0,0.126667,0.275000
44,0cbr,NaN,NaN,NaN,course-v1:MinesTelecom+04018+session03,0,1,"bonjour,\nj'ai le meme type de pb .\nje n arri...",28,0.80,1,-0.200000,0.300000
46,0cbr,NaN,NaN,NaN,MinesTelecom/04021S02/session02,1,0,beaucoup de temps passe sur le net pour compre...,26,0.35,0,0.043750,0.325000
57,123AZER,NaN,NaN,NaN,CNAM/01002S04/session04,1,0,"bonjour, nicolas 50 ans officier de sapeurs-po...",10,0.77,1,0.174286,0.278571
68,12nono12,NaN,NaN,NaN,MinesTelecom/04017S02/session02,0,1,"en fait dans le simulateur, le servo moteur se...",31,0.98,1,0.015000,0.175000


In [3]:
#drop des outliers
data = data[data.user != "EGo41"]
data = data[data.user != "bgaultier"]
data = data[data.user != "EllaHamonic"]
data = data[data.user != "stiphaen"]
data.shape

(5314, 13)

# II. Modèle

## 1 Classification

In [3]:
from pycaret.classification import *

In [5]:
data_class = data.drop(['user', 'course_id', 'corpus','grade'], axis=1)
data_class.head()

,gender,country,level_of_education,nb_threads,nb_comments,delai_1er_post,certificate_eligible,polarity,subjectivity
10,NaN,NaN,NaN,0,1,5,0,0.126667,0.275000
44,NaN,NaN,NaN,0,1,28,1,-0.200000,0.300000
46,NaN,NaN,NaN,1,0,26,0,0.043750,0.325000
57,NaN,NaN,NaN,1,0,10,1,0.174286,0.278571
68,NaN,NaN,NaN,0,1,31,1,0.015000,0.175000


In [6]:
s = setup(data_class, target = 'certificate_eligible')

,Description,Value
0,session_id,4566
1,Target,certificate_eligible
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(5314, 9)"
5,Missing Values,True
6,Numeric Features,5
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.6773,0.7028,0.8221,0.7039,0.7580,0.2819,0.2904,0.2120
ada,Ada Boost Classifier,0.6674,0.6892,0.8090,0.6984,0.7495,0.2623,0.2690,0.1130
lightgbm,Light Gradient Boosting Machine,0.6459,0.6754,0.7618,0.6932,0.7256,0.2291,0.2316,0.0630
rf,Random Forest Classifier,0.6410,0.6499,0.7697,0.6858,0.7252,0.2126,0.2156,0.2730
lr,Logistic Regression,0.6392,0.6764,0.9537,0.6386,0.7648,0.1052,0.1582,0.5190
et,Extra Trees Classifier,0.6357,0.6364,0.7570,0.6848,0.7188,0.2048,0.2069,0.2830
knn,K Neighbors Classifier,0.6203,0.6301,0.7356,0.6763,0.7045,0.1760,0.1772,0.0480
lda,Linear Discriminant Analysis,0.6193,0.6136,0.9891,0.6193,0.7617,0.0207,0.0658,0.0270
nb,Naive Bayes,0.6187,0.6047,0.9821,0.6201,0.7601,0.0240,0.0694,0.0120
ridge,Ridge Classifier,0.6182,0.0000,0.9860,0.6191,0.7606,0.0195,0.0574,0.0250


In [8]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [9]:
print(best)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=4566, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [10]:
predict_model(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.6821,0.7044,0.8195,0.7195,0.7663,0.2762,0.2819


,nb_threads,nb_comments,delai_1er_post,polarity,subjectivity,gender_f,gender_m,gender_not_available,country_AD,country_AU,...,level_of_education_hs,level_of_education_jhs,level_of_education_m,level_of_education_none,level_of_education_not_available,level_of_education_other,level_of_education_p,certificate_eligible,Label,Score
0,0.0,5.0,2.0,0.213214,0.290476,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.7196
1,0.0,1.0,13.0,0.083125,0.175000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,0.6167
2,0.0,1.0,14.0,0.150000,0.350000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.6832
3,1.0,2.0,3.0,0.101429,0.207143,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.7372
4,4.0,1.0,1.0,0.079950,0.262000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.6648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,0.0,1.0,67.0,0.166000,0.340000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.7395
1591,0.0,1.0,25.0,0.275000,0.100000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.6495
1592,2.0,9.0,4.0,0.066184,0.199815,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.8356
1593,2.0,0.0,23.0,-0.250000,0.500000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0.7375


In [11]:
# export model
save_model(best, 'data/my_best_pipeline')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='certificate_eligible',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None...
                                             learning_rate=0.1, loss='deviance',
                                             max_depth=3, max_features=None,
                                             max_leaf_nodes=None,
                                             

## 2 Regression

In [6]:
data_class = data.drop(['user', 'course_id', 'corpus','certificate_eligible'], axis=1)
data_class.head()

,gender,country,level_of_education,nb_threads,nb_comments,delai_1er_post,total_length,grade,polarity,subjectivity
10,NaN,NaN,NaN,0,1,5,554,0.38,0.126667,0.275000
44,NaN,NaN,NaN,0,1,28,213,0.80,-0.200000,0.300000
46,NaN,NaN,NaN,1,0,26,216,0.35,0.043750,0.325000
57,NaN,NaN,NaN,1,0,10,336,0.77,0.174286,0.278571
68,NaN,NaN,NaN,0,1,31,261,0.98,0.015000,0.175000


In [7]:
from pycaret.regression import *


In [13]:
s = pycaret.regression.setup(data_class, target = 'grade')

,Description,Value
0,session_id,6335
1,Target,grade
2,Original Data,"(5349, 10)"
3,Missing Values,True
4,Numeric Features,6
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3744, 55)"


In [14]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.2891,0.1140,0.3376,0.1463,0.2324,0.8022,0.2150
ada,AdaBoost Regressor,0.3068,0.1189,0.3448,0.1100,0.2353,0.8043,0.0500
lightgbm,Light Gradient Boosting Machine,0.2911,0.1202,0.3466,0.1001,0.2375,0.7952,0.0560
rf,Random Forest Regressor,0.2945,0.1225,0.3498,0.0834,0.2395,0.8093,0.6460
et,Extra Trees Regressor,0.2981,0.1291,0.3592,0.0340,0.2451,0.8146,0.4920
br,Bayesian Ridge,0.3191,0.1306,0.3613,0.0226,0.2475,0.8889,0.0150
ridge,Ridge Regression,0.3181,0.1306,0.3614,0.0222,0.2472,0.8877,0.0090
lr,Linear Regression,0.3184,0.1312,0.3621,0.0181,0.2476,0.8909,0.3020
lar,Least Angle Regression,0.3185,0.1313,0.3623,0.0171,0.2478,0.8931,0.0130
omp,Orthogonal Matching Pursuit,0.3225,0.1326,0.3641,0.0077,0.2498,0.9068,0.0080


In [15]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
print(best)

In [ ]:
predict_model(best)

# III. Prediction

In [4]:
# charger le modèle avec pycaret
loaded_model = load_model('data/my_best_pipeline')
print(loaded_model)

Transformation Pipeline and Model Successfully Loaded
Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(ml_usecase='classification',
                                      target='certificate_eligible')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         '...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dummify(target='certificate_eligible')),
                ('fix_perfect', Remove_100(target='certificate_eligib

In [7]:
# # Si besoin de le récupérer avec pickle
# filename = 'data/modele_redump.sav'
# pickle.dump(loaded_model, open(filename, 'wb'))
# print("modele enregistré")

modele enregistré


In [8]:
# # load the model from disk
# filename = 'data/modele_redump.sav'
# loaded_model = pickle.load(open(filename, 'rb'))
# print("modèle chargé")

modèle chargé


In [9]:
# admettons que ces éléments soient l'ensemble des données entrées par l'utilisateur dans le formulaire
# VERSION DEVRAIT VALIDER
gender = "f"
country = np.NaN
level_of_education = np.NaN
nb_threads = 0
nb_comments = 50
# corpus = "c'est un roc, c'est un pic, c'est un cap, que dis-je c'est un cap, c'est une péninsule!"
corpus = "c'est nul, je n'aime pas, je suis subjectif et négatif"
delai_1er_post = 1

In [21]:
# admettons que ces éléments soient l'ensemble des données entrées par l'utilisateur dans le formulaire
# VERSION NE DEVRAIT PAS VALIDER
gender = "m"
country = "FR"
level_of_education = "hs"
nb_threads = 1
nb_comments = 0
# corpus = "c'est un roc, c'est un pic, c'est un cap, que dis-je c'est un cap, c'est une péninsule!"
corpus = "c'est nul, je n'aime pas, je suis subjectif et négatif"
delai_1er_post = 50

In [18]:
# traitement du corpus
#lower case
corpus = corpus.lower()
# enlever les accents
corpus= unidecode(corpus)

# analyse de sentiments
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
blob = tb(corpus)

polarity = (blob.sentiment[0])
subjectivity = (blob.sentiment[1])

In [22]:
# create df
df_input = pd.DataFrame(columns=["gender", "country", "level_of_education", "nb_threads", "nb_comments", "delai_1er_post", "polarity", "subjectivity"])
df_input.loc[0] = [gender, country, level_of_education, nb_threads, nb_comments, delai_1er_post, polarity, subjectivity ]

df_input

,gender,country,level_of_education,nb_threads,nb_comments,delai_1er_post,polarity,subjectivity
0,m,FR,hs,1,0,10,0.1,0.2


In [23]:
# prediction avec le modele
y_pred = loaded_model.predict(df_input)
print(y_pred)

if y_pred == [1]:
    print("Cet utilisateur devrait valider le diplome")
else: 
    print("Cet utilisateur ne devrait pas valider le diplome")

[1]
Cet utilisateur devrait valider le diplome
